<a href="https://colab.research.google.com/github/Bibirara/Natural-Disaster-Shock/blob/main/Base%20Line%20Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install --upgrade quantecon
import matplotlib.pyplot as plt
import numpy as np
import quantecon as qe

from numba import njit, prange
%matplotlib inline

In [4]:
class Arellano_Economy:
    " Stores data and creates primitives for the Arellano economy. "

    def __init__(self,
            B_grid_size= 251,   # Grid size for bonds
            B_grid_min=-0.45,   # Smallest B value
            B_grid_max=0.45,    # Largest B value
            y_grid_size=51,     # Grid size for income
            β=0.953,            # Time discount parameter
            γ=2.0,              # Utility parameter
            r=0.017,            # Lending rate
            ρ=0.945,            # Persistence in the income process
            η=0.025,            # Standard deviation of the income process
            θ=0.282,            # Prob of re-entering financial markets
            def_y_param=0.969): # Parameter governing income in default

        # Save parameters
        self.β, self.γ, self.r, = β, γ, r
        self.ρ, self.η, self.θ = ρ, η, θ

        self.y_grid_size = y_grid_size
        self.B_grid_size = B_grid_size
        self.B_grid = np.linspace(B_grid_min, B_grid_max, B_grid_size)
        mc = qe.markov.tauchen(y_grid_size, ρ, η, 0, 3)
        self.y_grid, self.P = np.exp(mc.state_values), mc.P

        # The index at which B_grid is (close to) zero
        self.B0_idx = np.searchsorted(self.B_grid, 1e-10)

        # Output recieved while in default, with same shape as y_grid
        self.def_y = np.minimum(def_y_param * np.mean(self.y_grid), self.y_grid)

    def params(self):
        return self.β, self.γ, self.r, self.ρ, self.η, self.θ

    def arrays(self):
        return self.P, self.y_grid, self.B_grid, self.def_y, self.B0_idx